# 1. Introduction

Terrorism. A word that most of us associate with terrible memories and/or- experiences. Every day we are reminded through news or by first-hand experience that this kind of terrible action seems to occur more often than ever before. This begs the question. Has the number of terrorist attacks increased over the years?


Terrorism can be a controversial subject especially with regards to the current media climate and all the talk about fake news. Therefore, I'll let the data speak for itself and be very explicit when I deviate from this to share my own thoughts.

So, with this in mind, I'm going to explore the data and see if any trends emerge. I'll mainly try to answer the following questions:
- Has the number of attacks increased over the years?
- Which are the most affected continents, countries, cities?
- Relationship between happiness score and terrorism?

# 2. Data Exploration

# 3. Data Preparation

# 4. Conclusion